# Challenge 01 - Prompt Engineering

Quick tip! To view the Table of Contents for this Notebook in VS Code, in the "Explorer" tab, expand the "Outline" section. If using Azure AI Machine Learning Studio, click the hamburger icon in the top left > View > Table of Contents.

## 1. Parameter Experimentation
Let's first set up the Challenge. Load the API key and relevant Python libraries using the cells below.

### 1.1 Preparation
#### 1.1.1 Environment Setup


#### Create or Update the environment file ".env"
Create or update an environment file named as `.env` under the same folder with your notebooks.

We've included a sample environment file for your reference. You can find the `sample-env.txt` file in the Student `Resources.zip` file. Please feel free to make any modification as needed and rename the file as `.env`. 

**NOTE:** Make sure to keep the name of each model the same as the model deployment name you set in Azure OpenAI.

**NOTE:** Please be aware that some AOAI models are deprecated. You should create the ones available as needed instead. To complete this challenge, gpt-35-turbo is the only model you need.

For information on current models, see: [Azure OpenAI Service models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)

If you are using Azure ML Studio, if you want to change the model name in the environment file, please run the following code to make a change

In [ ]:
# os.environ[{MODEL YOU WANT TO CHANGE}] = {NEW MODEL NAME}
# print({MODEL NAME})

####  Set up Azure OpenAI Environment

In [1]:
import openai
import os #para archivos de sistemas operativo
import json  #Para consultas json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
#Lee la llave y se asigna a la propiedad api_key de openai
API_KEY = os.getenv("OPENAI_API_KEY")  
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Parameter Explanation
We specify the hyperparameters for the Azure OpenAI models within the helper functions. Users can tune the parameters according to different needs.


###### **Temperature**
Temperature ranges from 0 to 2.
Here is a quick breakdown of how it works:
- Low temperature (0 to 0.3): More focused, coherent, and conservative outputs. (no se arriesga)
- Medium temperature (0.3 to 0.7): Balanced creativity and coherence. (Se arriesga con palabras aleatorias que ahcen sentido)
- High temperature (> 0.7): Highly creative and diverse, but potentially less coherent.(Talvez lo que diga no tenga mucha logica)
ASi tu modificas la generacion de la diversidad de las palabras.

###### **Top_p**
Van generando palabra por palabra,, primero una , la que sigue, la que sigue.. entonces esta secuencia debe tener secuencia/logica
Tu puedes hacer que ese conjunto de palabras (diccionario de palabras) o frases que sigan, sea muy grande o mas pequeno.

Sets the probability mass cutoff for token sampling, affecting the breadth of options the AI considers. 
Higher values lead to more randomness, while lower values result in more focused outputs.

The “top_p” parameter is like a filter that controls how many different words or phrases the language model considers when it’s trying to predict the next word. If you set the “top p” value to 0.5, the language model will only consider the 50 most likely words or phrases that might come next. But if you set the “top p” value to 0.9, the language model will consider the 90 most likely words or phrases.

And as "top_p" and "temperature" performs similar job as hyperparameter, we usually only tune one of them instead of both.

###### **Max_tokens**
delimita, limita cuandtos tokes puede generar,, pero puede generar respuestas que se corten las respuestas. 
https://platform.openai.com/tokenizer

Max_tokens determine the maximum length of the generated text. By setting a limit, you can control how much text the LLM model will return, making sure it doesn't give too long of an answer.

###### **Frequency_penalty**
Para evitar que repita palabras. Con uno negativo  
con 2 positivo hay mas variedad, hay menos cantidad de repeticion de palabras.
Penaliza la repeticion de palabras.

Frequency_penalty makes sure that the text that is generated is varied by giving a penalty to tokens that have already been used in the response.

It ranges from -2.0 to 2.0, with higher values resulting in more diverse output.

Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

#### 1.1.3 Helper Function
Throughout this hack, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs.

prompt y modelo / 
engine - nombre del deployment model

**get_chat_completion** helps create the OpenAI response using the chat model of your choice.

**get_completion_from_messages** helps create the OpenAI response using the chat model of your choice, enabling chat history.


In [68]:
#Crea la funcion
#Aca le envias una cadena de texto
#El model=chat_model es opcional, si no se pone si invoca el la variable chat_model
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 1000,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [4]:
#Aca pasas un arreglo
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Try out helper functions

In [5]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

Why did the tomato turn red? Because it saw the salad dressing!


In [6]:
#Es una conversacion entre el usuario y el asistente, la idea es que el modelo trate de entender 
# de que va la conversacion o como debe responder
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side!


#### Student Tasks (open questions):
1. Create a completion function for a higher temperature between (0,1).
2. Create a completion function for a lower max_token for shorter response.
3. Create acompletion functions with 2 different diversity penalty values between (0,2).

Try out the completion functions you create on the previous case, compare the result you get.

In [11]:
# Try out a completion function for a higher temperature between (0,1)
messages =  [      
    {'role':'system', 'content':'Eres un abogado que explica los temas de derecho a clientes como si fueran ninos'},   
    {'role':'user', 'content':'¿Qué es la constitución?'}  
]

response = get_completion_from_messages(messages, temperature=.9)
print(response)

¡Hola! La Constitución es como las reglas de un juego muy importante que se llama "país". En ella se establecen los derechos y deberes de todos los ciudadanos y del gobierno, para que todo el mundo sepa cómo se deben comportar y cuáles son sus derechos protegidos. Es como el libro de reglas que todos debemos seguir para que el juego se desarrolle de forma justa y todos podamos convivir de manera pacífica. ¿Te quedó claro?


In [12]:
# Try out a completion function for a lower max_token for shorter response
def get_chat_completion2(prompt, model=chat_model, max_tokens=200):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = max_tokens,
        top_p = 1.0
    )
    return response.choices[0].message["content"]


get_chat_completion2(prompt="tell me a history", max_tokens=100)

"of the Roman Empire\n\nThe Roman Empire was one of the most powerful and influential empires in world history. It was founded in 27 BC by the Roman Republic's first emperor, Augustus, and lasted until the fall of the Western Roman Empire in 476 AD.\n\nThe Roman Empire was known for its military prowess, engineering feats, and cultural achievements. It expanded its territory through conquest and colonization, eventually encompassing much of Europe, the Middle East, and North Africa.\n\nUnder the rule of"

In [16]:
# Try out completion functions with 2 different diversity penalty values between (0,2)
# Try out a completion function for a lower max_token for shorter response
def get_chat_completion3(prompt, model=chat_model, max_tokens=200, penalty=0.5):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = max_tokens,
        top_p = 1.0,
        frequency_penalty = penalty
    )
    return response.choices[0].message["content"]


get_chat_completion3(prompt="tell me a short history of Mexico", penalty = 2)

'Mexico has a rich and complex history that dates back thousands of years. The first civilization to emerge in Mexico was the Olmec, who lived along the Gulf Coast from around 1200 BCE to 400 BCE. They were followed by other civilizations such as the Maya, Zapotec, and Toltec.\n\nIn 1519, Spanish conquistador Hernán Cortés arrived in Mexico with his army and began a campaign to conquer the Aztec Empire. After several years of fighting, he succeeded in capturing their capital city Tenochtitlan (now Mexico City) in 1521.\n\nFor over three centuries after this conquest, Mexico was ruled by Spain as part of its vast empire. During this time period there were many uprisings against Spanish rule including one led by Miguel Hidalgo y Costilla which sparked Mexican independence on September 16th ,1810.\n\nAfter gaining independence from Spain in 1821,Mexico struggled with political instability for much of its early history before settling'

### 1.2 System Message Engineering
Users can achieve the response from models in their desired tone through adjusting the system message.

#### 1.2.1 Change of Tone

In [18]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Why don't scientists trust atoms?

Because they make up everything!


In [19]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hark! Why does a chicken coop only have two doors? Because if it had four, it would be a sedan!


#### Student Task (open question):
Make the assistant tell the joke in the tone of your favorite character by editing the system message.


In [20]:
# Make the assistant tell the joke in the tone of your favorite character by editing the system message.

messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like AMLO Mexico President.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)


Bueno, bueno, bueno... ¿Cómo se llama el pez que hace música? El Atún-tún. Jajaja... ¿Qué le pareció?


#### 1.2.2 Remind the Company Name

In [23]:
#Si tu preguntas algo de mensajes anteriores, la ia no va a recordarlo, pk cada interaccion es una conversacion diferente.
#Para esto debes enviarle todo el contxto en la misma conversacion
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, Mel. As a chatbot, I don't have access to your personal information such as where you work. May I suggest double-checking your emails or employment documents to confirm the name of your company?


#### Student Task: 
Make the assistant give a reliable reminder of company name by giving context in the system message.

In [25]:
# Make the assistant give a reliable reminder of company name by giving context in system message.

messages =  [  
{'role':'system', 'content':'You are friendly chatbot. That works for users in Sabritas Company'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)


Sure Mel! You work for Sabritas Company. Sabritas is a Mexican snacks food company that sells its products mainly in Mexico but also in other parts of Central America, Spain, and the United States. Is there anything else I can help you with?


## 2. Iterative Prompting Principles
It is generally a good practice to perform iterative prompting so the model could generate the most appropriate response given the user's specification. 
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**



### 2.1 Write clear and specific instructions

#### Tactic 1: Delimiters

#### Student Task:
Use delimiters to clearly indicate distinct parts of the input

Delimiters can be anything like: ````, """, < >, `<tag> </tag>`, `:`

In [26]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [27]:
# Use delimiters to clearly indicate distinct parts of the input, and ask the model to summarize the text.
# Los apostrofres son para python, texto multilinea

#Basta con separar el texto de la instruccion
text = f""" 
### DATA ###
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
### REQUEST ###
    Summarize the previous text, in less than 30 words.
### SUMMARY ###
    """

get_chat_completion(text)

'The 2020 Summer Olympics, held in Tokyo, Japan, were postponed to 2021 due to the COVID-19 pandemic and held without spectators. New events were introduced, and the United States topped the medal count.'

In [29]:
prompt = f""" 
### DATA ###
{text}
### REQUEST ###
Summarize the previous text, in less than 15 words.
### SUMMARY ###
"""

get_chat_completion(prompt)

'2020 Summer Olympics held in Tokyo with new events and COVID-19 restrictions. USA topped medal count.'

#### Tactic 2: Summarization: specify word counts, extract information

#### Text to summarize

In [30]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Student Task:
Summarize with a focus on the special parts of the 2020 Summer Olympics and with a word limit

In [37]:
# Summarize with a focus on the special parts of the 2020 Summer Olympics and with a word limit
prompt = f"""
Write a short summary about the following text in less than 30 words, focusing in the results of Medals Winners, just country and medal type.
+++{game_review}+++
"""

get_chat_completion(prompt=prompt)

'The United States topped the medal count with 39 golds and 113 total medals, followed by China with 38 golds and 89 total medals. Host nation Japan finished third with a record 27 golds and 58 total medals. New events were introduced, and new sports added for just one Games, including baseball and softball, karate, sport climbing, surfing, and skateboarding.'

#### Student Task:
Try "extract" instead of "summarize"

In [38]:
# Try "extract" instead of "summarize"
prompt = f"""
Extract relevant information about the following text in less than 30 words, focusing in the results of Medals Winners, just country and medal type.
+++{game_review}+++
"""

get_chat_completion(prompt=prompt)

'The United States won the most gold and total medals at the 2020 Summer Olympics, followed by China. Japan finished third with a record number of gold and total medals. New sports were introduced, and several countries won their first-ever Olympic medals.'

#### Tactic 3: Inferring: ask for emotions, sentiment, or topics 

#### Student Task:
Identify types of emotions and sentiment (positive/negative) of the review below

Format in a JSON object

In [39]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [42]:
# Identify types of emotions and sentiment (positive/negative) of the review above. Format in a JSON object.

prompt = f"""
Identify types of emotions and sentiment (positive/negative) of the following review by person.
Give me the output in Json object.
+++ {review}
"""

get_chat_completion(prompt=prompt)

'{\n    "Philip Barker": {\n        "Emotion": "Positive",\n        "Sentiment": "Positive"\n    },\n    "Hashimoto": {\n        "Emotion": "Gratitude",\n        "Sentiment": "Positive"\n    },\n    "Dominic Patten": {\n        "Emotion": "Mixed",\n        "Sentiment": "Negative"\n    },\n    "Thomas Bach": {\n        "Emotion": "Cliché-filled",\n        "Sentiment": "Negative"\n    },\n    "Alan Tyres": {\n        "Emotion": "Strangeness",\n        "Sentiment": "Neutral"\n    }\n}'

#### Student Challenge:
Infer 3 topics of the story below

In [43]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [44]:
# Infer 3 topics of the story above. Deduce/detecta
prompt = f"""
Detect the 3 more relevant topics of the following story, present them in bullet points
+++ {story}
"""

get_chat_completion(prompt=prompt)


'- The 2020 Summer Olympics held in Tokyo, Japan, from 23 July to 8 August 2021\n- The event was postponed from 2020 due to the global COVID-19 pandemic and was held without official spectators\n- New events were introduced in existing sports, and new sports were added to the Olympic program, with the United States topping the medal count.'

#### Tactic 4: Transforming: specify target language and writing style, and ask for grammar check

#### Student Task:
Universal Translator

People all over the world want to know the Olympic game news in their native language. In this case, the news needs to be translated into different languages. Translate each news item below into both Korean and English.

In [45]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [46]:
# Write code here for student task
prompt = f"""
Translate the following news to Korean and English
+++ {news}
"""

get_chat_completion(prompt=prompt)


'Korean: \n팔레스타인은 2021년 6월 28일 FINA 포인트 시스템을 기반으로 개최되는 올림픽에 각각 한 명의 최상위 선수(성별별)를 보내기 위해 FINA로부터 보편적 초청을 받았습니다. \n\nEnglish: \nPalestine has received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, based on the FINA Points System of June 28, 2021. \n\nThe competition consists of two rounds: preliminaries and finals. The top 8 relay teams with the best preliminary results advance to the finals. If necessary, swimming races are used to break ties to advance to the next round.'

#### Student Task:
Tone Transformation

Writing can vary based on the intended audience. ChatGPT can produce different tones. Transform the following message into a business letter.

In [47]:
original_message = "David, it's John! OMG, the Olympic game is so crazy"

In [50]:
# Task: write code here for student task

business_msg = f"""
Write the following message in a more Professional Tone
+++ {original_message} +++
"""

get_chat_completion(prompt=business_msg)


'Dear David,\n\nI hope this message finds you well. I wanted to touch base with you regarding the Olympic games. The level of competition and excitement has been truly remarkable. \n\nBest regards,\nJohn'

#### Student Task:
Format Conversion

ChatGPT can translate between formats. The prompt should describe the input and output formats. Convert the following JSON data into HTML format.

In [51]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [53]:
# Write code here for student task
msg = f"""
Convert the following Json data in HTMLtable format.
+++ {data_json} +++
"""

get_chat_completion(prompt=msg)


'<table>\n  <tr>\n    <th>The 2020 Summer Olympics Opening Ceremony audience name list</th>\n  </tr>\n  <tr>\n    <td>name</td>\n    <td>email</td>\n  </tr>\n  <tr>\n    <td>Shyam</td>\n    <td>shyamjaiswal@gmail.com</td>\n  </tr>\n  <tr>\n    <td>Bob</td>\n    <td>bob32@gmail.com</td>\n  </tr>\n  <tr>\n    <td>Jai</td>\n    <td>jai87@gmail.com</td>\n  </tr>\n</table>'

#### Student Task:
Spellcheck and Grammar check the following text. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [54]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [55]:
# Write code here for student task

msg = f"""
Proofread and corrct the following text
+++ {check_text} +++
"""

get_chat_completion(prompt=msg)

'"Alongside the main emblem blue, the five other colors used in the branding of the 2020 Games are: Kurenai red, Ai blue, Sakura pink, Fuji purple, and Matsuba green. The competition has three rounds: heats, semifinals, and a final."'

#### Tactic 5: Expanding: customize the automated reply

#### Student Task:
Customize the automated reply to the following customer email.

The customer faced an issue while buying the Olympics game ticket.

In [56]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [57]:
# Write code here for student task
msg = f"""
Bases on the sentiment of the review, customize the email to the customer:

Sentiment: {sentiment} 
Review: {review}

Email:
"""

get_chat_completion(prompt=msg)

'Dear [Customer],\n\nWe are sorry to hear that you have not received your ticket for the "Men\'s 100 metre freestyle swimming" game. We understand how frustrating this must be for you and we apologize for any inconvenience caused.\n\nWe would like to assure you that we are looking into this matter and will do everything we can to resolve it as soon as possible. We will be in touch with you shortly to provide an update on the status of your ticket.\n\nOnce again, we apologize for any inconvenience caused and thank you for your patience.\n\nBest regards,\n[Your Name]'

#### Tactic 6: Chatbot: personalize conversations for specific tasks or behaviors

#### Student Task:
Create a conversation with chatbot to know where the 2020 Summer Olympics is held.

In [58]:
# Write code here for student task

prompt = "Where were the 2020 Summer Olympics held?"

get_chat_completion(prompt)

'The 2020 Summer Olympics were scheduled to be held in Tokyo, Japan, but were postponed to 2021 due to the COVID-19 pandemic.'

### 2.2 Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task
Sometimes you can help the model "slow down" and give more robust, detailed answers by specifying the steps it should take.

Let's ask for output in multiple specified formats.

In [60]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 

get_chat_completion(prompt)

'"Tokyo 2020 Olympics: A Historic Event Amidst a Pandemic"\n\nKeywords: Olympics, Tokyo, pandemic, new events, medal count\n\n<div>\n    <h1>Article Publishing Information</h1>\n    <table>\n        <tr>\n            <td>The 2020 Summer Olympics</td>\n            <td>Olympics, Tokyo, pandemic, new events, medal count</td>\n        </tr>\n    </table>\n</div>'

#### Tactic 2: Instruct the model to work out its own solution

There are two main methods we will demonstrate in this section to get the model to work through a problem instead of rushing to a conclusion, chain-of-thought and chaining. The strategies can lead to increased accuracy, detail, and the ability to work through complex challenges.


2.1 - Chain-of-thought prompting
- Ask the model to reason
- One-shot example
- Chatbot reasoning

2.2 - Chaining

Let's continue working with the Olympics dataset.

#### 2.2.1 Chain-of-Thought Prompting

Let's do a bit of math. GPT models occasionally don't do super well on solving direct math problems, so let's walk GPT through the problem.

Let's break down tasks into smaller pieces.

Read more about methods and whitepaper research here: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Tactic 1: You can start by specifically asking the model to simply think step-by-step.

In [61]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)

'Step 1: Identify the Olympic games held at Sydney\nThe Olympic games held at Sydney were the 2000 Summer Olympics.\n\nStep 2: Identify the most decorated individual athlete\nTo identify the most decorated individual athlete, we need to look at the medal count of all athletes who participated in the 2000 Summer Olympics. According to the official website of the International Olympic Committee (IOC), the medal count for the 2000 Summer Olympics can be found here: https://www.olympic.org/sydney-2000/medal-count.\n\nStep 3: Determine the athlete with the maximum medals\nAfter reviewing the medal count, the athlete with the most medals at the 2000 Summer Olympics was Michael Phelps, an American swimmer. Phelps won a total of 6 gold medals and 2 bronze medals, bringing his total medal count to 8.\n\nTherefore, the ANSWER is: Michael Phelps.'

In [62]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

'To solve this problem, we need to identify the top two countries who won the most gold medals in the 2020 Tokyo Olympics. According to the context, the United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Therefore, the top two countries are the United States and China.\n\nNext, we need to determine the time zone difference between the United States and China. The United States has multiple time zones, but we can use the Eastern Time Zone as a reference since it is the most populous. The Eastern Time Zone is 13 hours behind China Standard Time (CST). Therefore, the largest time zone difference between the United States and China is 13 hours.\n\nTherefore, the answer to the problem is that the largest time zone difference is 13 hours between the top two countries who won the most gold medals in the 2020 Tokyo Olympics, which are the United States and China.'

#### Tactic 2: One-shot example 
Another common tactic is to provide one example of a query and an ideal response. The model will learn from that example and apply the patterns to a new question.

In [63]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

'The United States has 74 medals that are not gold medals (113 total medals - 39 gold medals). \n\nGreat Britain has 42 medals that are not gold medals (64 total medals - 22 gold medals). \n\nTherefore, the United States has 32 more non-gold medals than Great Britain (74 - 42 = 32).'

In [64]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

'[Step 1] - There are three types of medals: gold, silver, and bronze\n[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (38) and total (89) counts. \n            89-38=51, so there are 51 silver and bronze medals combined. The answer is 51.'

#### Tactic 3: Chatbot chain-of-thought reasoning
You can build in logic using variables so the chatbot can dynamically demonstrate specific ways of thinking about a problem.

**The input place is at the top of the screen. You should see a popup. Type 'quit' if you want to exit.**

In [65]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


1

Hello! I am a decision bot. What is the problem, scenario, or question you need help with?


Great! Let's start with step 1. What is the key element or variable in your question about "Familia"?

4

I'm sorry, I'm not sure I understand. Can you please provide more context or clarify your question about "Familia"?

6

Thank you for clarifying. Let's begin with step 1:

Key element: Relationship with your in-laws.

What is your current relationship with your in-laws like?

8

I'm sorry, I didn't receive a response. Can you please let me know what your current relationship with your in-laws is like?

10

No problem. Let's move on to step 2:

Relationship/connection: Your behavior towards your in-laws and their behavior towards you.

How do you typically behave around your in-laws? How do they behave around you?

12

I'm sorry, I didn't receive a response. Can you please let me know how you typically behave around your in-laws and how they behave around you? This will help us move on to

#### 2.2.2 - Chaining
Similar to some earlier examples, you can use model outputs from previous queries into other queries. We will show you later in the Hack how to do this at scale.

In [66]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary
#La salida de una conversacion es la entrada o parte de la entrada para la siguinte.

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

The 2020 Tokyo Olympics have come to an end, and the medal count is in. The United States came out on top with 113 total medals, including 39 golds. China was a close second with 89 total medals, including 38 golds. Japan, the host country, finished third with 58 total medals, including 27 golds.

The winners of the Olympics are undoubtedly the United States, China, and Japan. These countries dominated the medal count and showcased their athletic prowess on the world stage. The United States, in particular, had a strong showing in swimming, track and field, and gymnastics, among other sports.

However, there were also some surprising winners, such as Bermuda, the Philippines, and Qatar, who each won their first-ever gold medals. These countries may not have dominated the medal count, but their victories were significant and inspiring.

On the other hand, there were also some losers of the Olympics. Countries like Burkina Faso, San Marino, and Turkmenistan did not win any gold medals an

#### Student Task (Chaining): 

Your job is now to write code that will determine the country that won the most silver and bronze medals combined.

We can see that the model performs poorly on answering the question directly.

In [80]:
prompt = f"""
Based on the context, which country had the most silver and bronze medals?

Make a table of the medal winners by country and medal type

Context: {text}
"""

num_medals = get_chat_completion(prompt)

prompt = f"""
    Tell me which country had the most silver and bronze medals?
    {num_medals}
"""

get_chat_completion(prompt)

'The United States had the most silver and bronze medals with 41 and 33 respectively.'

Write prompts in the cell below that will help the model answer the question by breaking down the tasks into different steps.

You should be able to get the model to answer the question in 2-3 steps.

## Conclusion

In this first challenge, we covered how to interact with the Azure OpenAI for different goals. Hopefully you were able to see the dynamic versatility of the models and how they can be used to solve a variety of problems using different techniques.

We gave the API short pieces of text using fixed variables. In the next set of challenges, you will see how to use the API with larger datasets.